### Jim Xu
### 831-156-383

# Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening

Still being developed, but you may get started on this when you are finished with Assignment 3.

# Table of Contents
* [Assignment 4: Negamax with Alpha-Beta Pruning and Iterative Deepening](#Assignment-4:-Negamax-with-Alpha-Beta-Pruning-and-Iterative-Deepening)
	* [Initial Code](#Initial-Code)
	* [Add moves counter](#Add-moves-counter)
	* [negamaxIDS](#negamaxIDS)
	* [negamaxIDSab](#negamaxIDSab)
	* [Grading](#Grading)
	* [Extra Credit](#Extra-Credit)


For this assignment, you will investigate the advantages of alpha-beta
pruning applied to Tic-Tac-Toe.  To do so, follow these steps.

## Initial Code <font color='red'>UPDATED Oct 8</font>

In [1]:
# CITED FROM https://piazza.com/class/j34vndmsdl75hu?cid=136
def ebf(nNodes, depth, precision=0.01):
    if nNodes == 0:
        return 0

    def ebfRec(low, high):
        mid = (low + high) * 0.5
        if mid == 1:
            estimate = 1 + depth
        else:
            estimate = (1 - mid**(depth + 1)) / (1 - mid)
        if abs(estimate - nNodes) < precision:
            return mid
        if estimate > nNodes:
            return ebfRec(low, mid)
        else:
            return ebfRec(mid, high)

    return ebfRec(1, nNodes)

In [2]:
def negamax(game, depthLeft):
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamax(game, depthLeft-1)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
    return bestValue, bestMove

In [3]:
class TTT(object):

    def __init__(self):
        self.board = [' ']*9
        self.player = 'X'
        if False:
            self.board = ['X', 'O', 'O', 'X', ' ', ' ', ' ', ' ', ' ']
            self.player = 'X'
        self.playerLookAHead = self.player
        # record the nodes explored
        self.movesExplored = 0

    def setTTT(self, board, player):
        self.board = board
        self.player = player
        
    def locations(self, c):
        return [i for i, mark in enumerate(self.board) if mark == c]

    def getMoves(self):
        moves = self.locations(' ')
        return moves

    def getUtility(self):
        whereX = self.locations('X')
        whereO = self.locations('O')
        wins = [[0, 1, 2], [3, 4, 5], [6, 7, 8],
                [0, 3, 6], [1, 4, 7], [2, 5, 8],
                [0, 4, 8], [2, 4, 6]]
        isXWon = any([all([wi in whereX for wi in w]) for w in wins])
        isOWon = any([all([wi in whereO for wi in w]) for w in wins])
        if isXWon:
            return 1 if self.playerLookAHead is 'X' else -1
        elif isOWon:
            return 1 if self.playerLookAHead is 'O' else -1
        elif ' ' not in self.board:
            return 0
        else:
            return None  ########################################################## CHANGED FROM -0.1

    def isOver(self):
        return self.getUtility() is not None

    def makeMove(self, move):
        self.movesExplored += 1
        self.board[move] = self.playerLookAHead
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board[move] = ' '
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def __str__(self):
        s = '{}|{}|{}\n-----\n{}|{}|{}\n-----\n{}|{}|{}'.format(*self.board)
        return s
    
    def getNumberMovesExplored(self):
        return self.movesExplored

Check that the following function `playGame` runs
correctly. Notice that we are using *negamax* to find the best move for
Player X, but Player O, the opponent, is using function *opponent*
that follows the silly strategy of playing in the first open position.

In [4]:
def opponent(board):
    return board.index(' ')

def playGame(game,opponent,depthLimit):
    print(game)
    while not game.isOver():
        score,move = negamax(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'to', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = opponent(game.board)
            game.makeMove(opponentMove)
            print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()
    print(game.getNumberMovesExplored())

In [5]:
game = TTT()
playGame(game,opponent,20)

 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 
558334


## Add moves counter

Evaluate the efficiency of the search by keepting track of the number of nodes explored, which is the same as the number of moves explored, during a game. Do this by adding a counter named `movesExplored` to the `TTT` constructor where it is initialized to 0 and increment the counter in the `TTT.makeMove` method.  Add a method `ttt.getNumberMovesExplored()` to get its current value.  So

    print('Number of moves explored', game.getMovesExplored())
    
will print the number of moves explored. You will not use a global variable for counting this time.

## negamaxIDS 

<font color='red'>UPDATED Oct 4</font>

Write a new function named `negamaxIDS` that performs an iterative deepening negamax search.  Replace the first line in the `while` loop of `playGame` with

        score,move = negamaxIDS(game,depthLimit)
        
where `depthLimit` is now the maximum depth and multiple `negamax` searches are performed for depth limits of $1, 2, \ldots,$ maximum depth.

But, when should you stop?  Can you stop before readhing the depthLimit?  If not, there is no point to doing iterative deepening.

For Tic-Tac-Toe, we can stop as soon as a call to `negamax` returns a winning move.  This will have a value of 1 for Tic-Tac-Toe.  To keep our `negamaxIDS` function general, add a method called `getWinningValue` to the `TTT` class that just returns 1.  Then `negamaxIDS` can call `game.getWinningValue()` to determine the value of a winning move for this game.  If the maximum depth is reached and no winning move has been found, return the best move found over all depth limts.

In [6]:
# from A4mysolution import *
def negamaxIDS(game, depthLimit):
    bestValue = -float('infinity')
    bestMove = None
    for i in range (1, depthLimit+1):
        value, move = negamax(game, i)
        if value is None:
            continue
        if bestValue < value:
            bestValue = value
            bestMove = move
        if bestValue == 1:
            return bestValue, bestMove
    return bestValue, bestMove

## negamaxIDSab

Now for the hardest part.  Make a new function `negamaxIDSab` by duplicating `negamaxIDS` and add the code to implement alpha-beta pruning.

In [7]:
# from A4mysolution import *
def negamaxIDSab(game, depthLimit, alpha=-float('infinity'), beta=float('infinity')):
    bestValue = -float('infinity')
    bestMove = None
    for i in range (1, depthLimit+1):
        value, move = negamaxab(game, i, alpha, beta)
        if value is None:
            continue
        if bestValue < value:
            bestValue = value
            bestMove = move
        if bestValue == 1:
            return bestValue, bestMove
    return bestValue, bestMove

def negamaxab(game, depthLeft, alpha=-float('infinity'), beta=float('infinity')):
    # In the for loop for trying moves, negate and swap the values of alpha and beta, 
    # and the returned value from recursive calls must be negated.
    alpha, beta = -beta, -alpha
    # If at terminal state or depth limit, return utility value and move None
    if game.isOver() or depthLeft == 0:
        return game.getUtility(), None # call to negamax knows the move
    # Find best move and its value from current state
    bestValue, bestMove = None, None
    for move in game.getMoves():
        # Apply a move to current state
        game.makeMove(move)
        # Use depth-first search to find eventual utility value and back it up.
        #  Negate it because it will come back in context of next player
        value, _ = negamaxab(game, depthLeft-1, alpha, beta)
        # Remove the move from current state, to prepare for trying a different move
        game.unmakeMove(move)
        if value is None:
            continue
        value = - value
        if bestValue is None or value > bestValue:
            # Value for this move is better than moves tried so far from this state.
            bestValue, bestMove = value, move
        # Update alpha to maximum of bestValue and current alpha.
        alpha = max(bestValue, alpha)
        if alpha >= beta:
            # Do early return if bestScore is greater than or equal to beta.
            return bestValue, bestMove
    return bestValue, bestMove

## playGames

Now duplicate the game playing loop so three complete tic-tac-toe games are played.  Call this new version `playGames`. For the first game, use `negamax`. For the second game, use `negamaxIDS`.  For the third game, use `negamaxIDSab`.  At the end of each game, print the number of X's in the final board, the number moves explored, the depth of the game which is the number of moves made by X and O, and the effective branching factor.  When you run `playGames` you should see the tic-tac-toe positions after each move and, after all games are done, a line for each game like the following lines, which were <font color='red'>UPDATED Oct 8</font>.

    negamax made 4 moves. 558334 moves explored for ebf(558334, 7) of 6.46
    negamaxIDS made 3 moves. 23338 moves explored for ebf(23338, 5) of 7.26
    negamaxIDSab made 3 moves 6053 moves explored for ebf(6053, 5) of 5.48

Your results may be different. 

The value of the depth is the total number of moves made by X and by O during the search.  You can calculate this by keeping a list of all board states, or by just counting the number of X's and O's in the final board.

In [8]:
names = ["negamax", "negamaxIDS", "negamaxIDSab"]
functions = [negamax, negamaxIDS, negamaxIDSab]
steps = []
for i, function in enumerate(functions):
    print(i)
    steps.append(i)
a = (1,2,3,4)
a[2]

0
1
2


3

In [9]:
def playGames(opponent, depthLimit):
    names = ["negamax", "negamaxIDS", "negamaxIDSab"]
    functions = [negamax, negamaxIDS, negamaxIDSab]
    myGames = []
    for i, function in enumerate(functions):
        print("\n", names[i], ":")
        game = TTT()
        print(game)
        myStep = 0
        while not game.isOver():
            score,move = function(game,depthLimit)
            if move == None :
                print('move is None. Stopping.')
                break
            myStep += 1
            game.makeMove(move)
            print('Player', game.player, 'to', move, 'for score' ,score)
            print(game)
            if not game.isOver():
                game.changePlayer()
                opponentMove = opponent(game.board)
                game.makeMove(opponentMove)
                print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
                print(game)
                game.changePlayer()
        myGame = (names[i], myStep, game.getNumberMovesExplored(), myStep*2-1)
        myGames.append(myGame)
    for myGame in myGames:
        print(myGame[0], " made ",  myGame[1], " moves. ", myGame[2],
              " moves explored for ebf(", myGame[2], ",", myGame[3], ") of ", '{:.2f}'.format(ebf(myGame[2], myGame[3])))

Here are some example results. <font color='red'>Updated October 8, 3:15pm </font>

In [10]:
playGames(opponent, 10)


 negamax :
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 4 for score 1
X|O|O
-----
X|X| 
-----
 | | 
Player O to 5
X|O|O
-----
X|X|O
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X|X|O
-----
X| | 

 negamaxIDS :
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| | 

 negamaxIDSab :
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 1
X| | 
-----
 | | 
-----
 | | 
Player O to 1
X|O| 
-----
 | | 
-----
 | | 
Player X to 3 for score 1
X|O| 
-----
X| | 
-----
 | | 
Player O to 2
X|O|O
-----
X| | 
-----
 | | 
Player X to 6 for score 1
X|O|O
-----
X| | 
-----
X| |

## Grading

As always, download and extract from [A4grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A4grader.tar)

In [11]:
%run -i A4grader.py


Testing negamax starting from ['O', 'X', ' ', 'O', ' ', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of 1

--- 10/10 points. negamax correctly explored 124 states.

Testing negamax starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamax correctly returns value of -1 and move of 5

Testing negamaxIDS with max depth of 5, starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 10/10 points. negamaxIDS correctly returns value of -1 and move of 5

Testing negamaxIDSab starting from ['O', 'X', 'X', 'O', 'O', ' ', ' ', 'X', ' ']

--- 20/20 points. negamaxIDSab correctly returns value of -1 and move of 5

Testing playGame with opponent that always plays in highest numbered position.
 | | 
-----
 | | 
-----
 | | 
Player X to 0 for score 0
X| | 
-----
 | | 
-----
 | | 
Player O to 8
X| | 
-----
 | | 
-----
 | |O
Player X to 2 for score 1
X| |X
-----
 | | 
-----
 | |O
Player O to 7
X| |X
-----
 | | 
-----
 |O|O
Player X to 1 for 

## Extra Credit 

Earn one extra credit point for each of the following.

  - Implement another game and repeat the above steps.

  - Implement a random move chooser as the opponent (Player O) and determine how many times Player X can win against this opponent as an average over multiple games.

### Description

**Get Largest Number** is game where two players in turn add a value to the total value and try to get the a limited maximum value. The total value starts from 0, and the limit for addition value is called stepLimit, the maximum value we try to achieve is called upperLimit.

For example, stepLimit is 2, and the upperLimit is 10. Two players, namely 'X' and 'O', one of the possible way is:
- total is: 0
- 'X' adds 2
- total is: 2
- 'O' adds 1
- total is: 3
- 'X' adds 1
- total is: 4
- 'O' adds 2
- total is: 6
- 'X' adds 1
- total is: 7
- 'O' adds 2
- total is: 9
- 'X' adds 1
- total is: 10

Then 'X' comes to the maximum value and wins the game

#### For the following discussion, we set stepLimit is 3, and the upperLimit is 21

In [12]:
# Get Largest Number
class GLN(object):
    def __init__(self, upperLimit = 21, stepLimit = 3):
        self.stepLimit = stepLimit
        self.upperLimit = upperLimit
        self.board = 0
        self.player = 'X'
        self.playerLookAHead = self.player
        self.movesExplored = 0

    def setGLN(self, board, player):
        self.board = board
        self.player = player

    def getMoves(self):
        return [ i for i in range(1, min(self.upperLimit-self.board, self.stepLimit)+1) ]

    def getUtility(self):
        if self.board == self.upperLimit:
            return -1
        elif self.board < self.upperLimit:
            return 0
        else:
            return None  ########################################################## CHANGED FROM -0.1

    def isOver(self):
        return self.getUtility() is not 0

    def makeMove(self, move):
        self.movesExplored += 1
        self.board = self.board + move
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def changePlayer(self):
        self.player = 'X' if self.player == 'O' else 'O'
        self.playerLookAHead = self.player

    def unmakeMove(self, move):
        self.board = self.board - move
        self.playerLookAHead = 'X' if self.playerLookAHead == 'O' else 'O'

    def __str__(self):
        return "total: " + str(self.board)
    
    def getNumberMovesExplored(self):
        return self.movesExplored

In [13]:
import random

# Implement a random move chooser as the opponent (Player O)
def GLNopponent(game):
    return random.randint(1,min(game.upperLimit-game.board, 3))

def playGLNGame(game,opponent,depthLimit):
    print(game)
    print(game.isOver())
    while not game.isOver():
        score,move = negamax(game,depthLimit)
        if move == None :
            print('move is None. Stopping.')
            break
        game.makeMove(move)
        print('Player', game.player, 'add', move, 'for score' ,score)
        print(game)
        if not game.isOver():
            game.changePlayer()
            opponentMove = GLNopponent(game)
            game.makeMove(opponentMove)
            print('Player', game.player, 'add', opponentMove)   ### FIXED ERROR IN THIS LINE!
            print(game)
            game.changePlayer()
    print(game.getNumberMovesExplored())

Lets look some Examples on Get Largest Number game:
1. if it is your turn and the total score is 17, there is no chance for you to win, since the only possible value you can achieve is (18, 19, 20), and for the opponent, he can easily achieve 21 from any of those values. As a result, negamax returned score -1 and move 1

In [14]:
game = GLN()
board = 17
player = 'X'
game.setGLN(board, player)
negamax(game, 10)

(-1, 1)

2.if it is your turn and the total score is 16, you can make total to 17 and then back to example 1, your opponent has no chance to win. As a result, negamax returned score 1 and move 1

In [15]:
game = GLN()
board = 16
player = 'X'
game.setGLN(board, player)
negamax(game, 10)

(1, 1)

3.if it is your turn and the total score is 15, you can make total to 17 and then back to example 1, your opponent has no chance to win. As a result, negamax returned score 1 and move 2

In [16]:
game = GLN()
board = 15
player = 'X'
game.setGLN(board, player)
negamax(game, 10)

(1, 2)

In [17]:
def playGLNGames(opponent, depthLimit, iterFlag=False, iterTimes=1):
    names = ["negamax", "negamaxIDS", "negamaxIDSab"]
    functions = [negamax, negamaxIDS, negamaxIDSab]
    myGames = []
    myWins = []
    for i, function in enumerate(functions):
        print("\n", names[i], ":")
        winTimes = 0        
        for times in range(iterTimes):
            game = GLN()
            if(times == iterTimes -1): print(game)
            myStep = 0
            while not game.isOver():
                score,move = function(game,depthLimit)
                if move == None :
                    if(times == iterTimes -1): print('move is None. Stopping.')
                    break
                myStep += 1
                game.makeMove(move)
                if(times == iterTimes -1): print('Player', game.player, 'to', move, 'for score' ,score)
                if(times == iterTimes -1): print(game)
                if not game.isOver():
                    game.changePlayer()
                    opponentMove = GLNopponent(game)
                    game.makeMove(opponentMove)
                    if(times == iterTimes -1): print('Player', game.player, 'to', opponentMove)   ### FIXED ERROR IN THIS LINE!
                    if(times == iterTimes -1): print(game)
                    game.changePlayer()
            if game.isOver() and game.player == 'X':
                winTimes += 1
            if(times == iterTimes -1): myGame = (names[i], myStep, game.getNumberMovesExplored(), myStep*2-1)
            if(times == iterTimes -1): myGames.append(myGame)
        myWins.append(winTimes)
    for myGame in myGames:
        print(myGame[0], " made ",  myGame[1], " moves. ", myGame[2],
              " moves explored for ebf(", myGame[2], ",", myGame[3], ") of ", '{:.2f}'.format(ebf(myGame[2], myGame[3])))
    if iterFlag is True:
        for i, name in enumerate(names):
            print(name, " produces win rate ", myWins[i]/iterTimes)
    
    
def playMultiGLNGames(opponent, depthLimit, iterTimes=50):
    return playGLNGames(opponent, depthLimit, True, iterTimes)

In [18]:
playGLNGames(GLNopponent, 10)


 negamax :
total: 0
Player X to 1 for score 0
total: 1
Player O to 3
total: 4
Player X to 1 for score 1
total: 5
Player O to 3
total: 8
Player X to 1 for score 1
total: 9
Player O to 2
total: 11
Player X to 2 for score 1
total: 13
Player O to 3
total: 16
Player X to 1 for score 1
total: 17
Player O to 2
total: 19
Player X to 2 for score 1
total: 21

 negamaxIDS :
total: 0
Player X to 1 for score 0
total: 1
Player O to 3
total: 4
Player X to 1 for score 1
total: 5
Player O to 2
total: 7
Player X to 2 for score 1
total: 9
Player O to 2
total: 11
Player X to 2 for score 1
total: 13
Player O to 3
total: 16
Player X to 1 for score 1
total: 17
Player O to 3
total: 20
Player X to 1 for score 1
total: 21

 negamaxIDSab :
total: 0
Player X to 1 for score 0
total: 1
Player O to 1
total: 2
Player X to 3 for score 1
total: 5
Player O to 3
total: 8
Player X to 1 for score 1
total: 9
Player O to 3
total: 12
Player X to 1 for score 1
total: 13
Player O to 2
total: 15
Player X to 2 for score 1
total:

In [19]:
playMultiGLNGames(GLNopponent, 10)


 negamax :
total: 0
Player X to 1 for score 0
total: 1
Player O to 3
total: 4
Player X to 1 for score 1
total: 5
Player O to 1
total: 6
Player X to 3 for score 1
total: 9
Player O to 2
total: 11
Player X to 2 for score 1
total: 13
Player O to 3
total: 16
Player X to 1 for score 1
total: 17
Player O to 2
total: 19
Player X to 2 for score 1
total: 21

 negamaxIDS :
total: 0
Player X to 1 for score 0
total: 1
Player O to 1
total: 2
Player X to 3 for score 1
total: 5
Player O to 2
total: 7
Player X to 2 for score 1
total: 9
Player O to 1
total: 10
Player X to 3 for score 1
total: 13
Player O to 3
total: 16
Player X to 1 for score 1
total: 17
Player O to 3
total: 20
Player X to 1 for score 1
total: 21

 negamaxIDSab :
total: 0
Player X to 1 for score 0
total: 1
Player O to 3
total: 4
Player X to 1 for score 1
total: 5
Player O to 1
total: 6
Player X to 3 for score 1
total: 9
Player O to 2
total: 11
Player X to 2 for score 1
total: 13
Player O to 3
total: 16
Player X to 1 for score 1
total: